In [1]:
# 1. 请先使用上位机升级机器人电机通信固件版本到0.3.12.25 以上，驱动固件版本升级到0.2.10.20 以上 详细使用请参考上位机使用说明
# 2. 请使用上位机对机器人零位进行标定,无绝编可不标
# 3. 请使用上位机调整机器人电机pd参数（注意不是pid参数），参考值：上肢（头腰双臂）p：100，d：0，可以根据软硬自己调节
# 4. 请使用指定的docker镜像进行开发，请使用开源仓库中的docker启动脚本启动docker

In [2]:
# fourier_hardware_py 用于对机器人进行分组控制
import fourier_hardware_py
# grx_sot_py 用于对机器人进行运动学，动力学求解
import grx_sot_py

In [3]:
# 加载对应机型的config配置文件
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/grmini_t1")

In [4]:
# 配置上肢控制组在ft模式（注意ft模型的控制组必须在电机enable之后立刻发送力矩命令），配置下肢控制组为pd模式
config.changeControlGroupMotorMode("left_manipulator","ft")
config.changeControlGroupMotorMode("right_manipulator","ft")
config.changeControlGroupMotorMode("waist","pd")
config.changeControlGroupMotorMode("left_leg","pd")
config.changeControlGroupMotorMode("right_leg","pd")
config.changePubState(True)

True

In [ ]:
# 此处会尝试与机器人建立连接，如果失败（存在control group无法连接），将会持续报错，请先检查机器人关节ip是否可以ping通，
# 如果不能ping通，请尝试重启机器人或寻求硬件支持
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/grmini_t1",config)

In [6]:
# 根据配置文件定义对应的机器人类，支持运动学、动力学求解
robot = grx_sot_py.RobotWrapper("/usr/local/config/fourier_hardware/grmini_t1", grx_sot_py.Flags.ignore_collisions)

In [7]:
# 定义机器人对应的控制组和与控制组匹配的关节顺序，输出对应的字典信息
group_names = []
group_joints = []
group_joints_pos_order = {}
group_joint_vel_order = {}

for group_name in config.getControlGroupNames():
    if config.getControlGroupMotorMode(group_name)== "ignore":
        continue
    group_names.append(group_name)
    robot_joint_names = hardware.getControlGroupJointNames(group_name)
    group_joints.append(robot_joint_names)
    joint_pos_orders = []
    joint_vel_orders = []
    for robot_joint_name in robot_joint_names:
        joint_pos_orders.append(robot.get_joint_offset(robot_joint_name))
        joint_vel_orders.append(robot.get_joint_v_offset(robot_joint_name))
    group_joints_pos_order[group_name]= joint_pos_orders
    group_joint_vel_order[group_name] = joint_vel_orders

In [8]:
print(config.getRobotName())
print(config.getControlGroupNames())

grmini_t1
['waist', 'left_manipulator', 'right_manipulator', 'left_leg', 'right_leg']


In [ ]:
result = hardware.enableControlGroup("waist")
reuslt = hardware.enableControlGroup("left_leg") and result 
result = hardware.enableControlGroup("right_leg") and result
# result 为 true 代表控制组正常使能
print(result)

In [10]:
# 获取机器人当前状态下的对应控制组['waist', 'head', 'left_manipulator', 'right_manipulator', 'left_leg', 'right_leg']的关节qpos(rad),
ll = hardware.getControlGroupState("left_leg")
rl = hardware.getControlGroupState("right_leg")


In [11]:
# 生成机器人双腿关节从当前位置到零位的轨迹
import numpy as np
leg_init = [-0.312, 0.0, 0.0, 0.669, 0.0, -0.363]
ll_traj = fourier_hardware_py.bridgeTrajectory(ll.q[-6:],leg_init)
rl_traj = fourier_hardware_py.bridgeTrajectory(rl.q[-6:],leg_init)

In [12]:
# 执行回零轨迹，驱动左腿控制组关节回到零位
import time

# for i in range(len(rm_traj[1])):
#     start_time = time.perf_counter()
#     # 获取机器人真机的状态信息
#     state = hardware.readRealRobotState()
#     # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
#     state.qd = np.zeros((state.qd).size)
#     state.qdd = np.zeros((state.qdd).size)
#     # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
#     regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
#     params = robot.get_inertia_params()
#     gravity_torque = regressor.dot(params)
#     hardware.setControlGroupPosCmd("right_manipulator",rm_traj[1][i],np.zeros(5),gravity_torque[group_joint_vel_order["right_manipulator"]])
#     state = hardware.readRealRobotState()

#     end_time = time.perf_counter()
#     #计算控制循环的周期，保持控制频率的稳定
#     execution_time = (end_time - start_time)
#     if  execution_time < 1.0/400 :
#         time.sleep(0.0025-execution_time)
#     else:
#         print("control loop is slow",execution_time)

for i in range(len(ll_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    hardware.setControlGroupPosCmd("left_leg",ll_traj[1][i])
    state = hardware.readRealRobotState()

    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [13]:
# 驱动右腿控制组关节回到零位
import time

# for i in range(len(lm_traj[1])):
#     start_time = time.perf_counter()
#     # 获取机器人真机的状态信息
    
#     state = hardware.readRealRobotState()
#     # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
#     state.qd = np.zeros((state.qd).size)
#     state.qdd = np.zeros((state.qdd).size)
#     # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
#     regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
#     params = robot.get_inertia_params()
#     gravity_torque = regressor.dot(params)
#     hardware.setControlGroupPosCmd("left_manipulator",lm_traj[1][i])
#     state = hardware.readRealRobotState()
    
#     end_time = time.perf_counter()
#     #计算控制循环的周期，保持控制频率的稳定
#     execution_time = (end_time - start_time)
#     if  execution_time < 1.0/400 :
#         time.sleep(0.0025-execution_time)
#     else:
        # print("control loop is slow",execution_time)
for i in range(len(rl_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    hardware.setControlGroupPosCmd("right_leg",rl_traj[1][i])
    state = hardware.readRealRobotState()

    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if execution_time < 1.0/400:
        time.sleep(0.0025 - execution_time)
    else:
        print("control loop is slow", execution_time)

In [26]:
# 设置控制组临时pd参数
result = hardware.setControlGroupPIDParams("left_leg",[100,100,100,100,20,10],[0,0,0,0,0,0],[0,0,0,0,0,0])
result = hardware.setControlGroupPIDParams("waist",[10],[0],[1]) and result
result = hardware.setControlGroupPIDParams("right_leg",[100,100,100,100,20,10],[0,0,0,0,0,0],[0,0,0,0,0,0]) and result
# 根据result结果可以判定是否设置成功
print(result)

True


In [15]:
# 尝试切换控制组的控制模式
result = hardware.changeGroupMode("waist", fourier_hardware_py.MotorMode.kPosition)
print(result)

True
[2025-05-14 14:58:54] [19333] [error] 192.168.137.90 Get subscribe data failed with code: -302


In [24]:
hardware.setControlGroupPosCmd("waist",[0.1])

True

In [22]:
hardware.setControlGroupPIDParams("waist",[0.1],[0.0001],[0.13])

True

In [25]:
result = hardware.changeGroupMode("waist", fourier_hardware_py.MotorMode.kPD)
print(result)

True


In [ ]:
# 使能整个机器人
import numpy as np
import math
import time
ERROR_CODE = 50 #关节组报错的错误码
hardware.enableControlGroup("left_manipulator")
hardware.enableControlGroup("right_manipulator")
while (True):
    start_time = time.perf_counter()

    # 返回一个字典，字典的key是control group 的名字，value是连续失去连接的次数（最大观测值是50），如果value等于50，代表该control group 失去连接
    # 应该在程序控制周期内实时观测此值，保证程序运行的安全
    motor_stats = hardware.getGroupsLoss()
    #检查机器人真机的硬件状态，如果机器人使能后某个电机断连，将会切断控制循环命令，并答应错误信息。
    if ERROR_CODE in motor_stats.values():
        print(f"motor get error.")
        break

    # 获取机器人真机的状态信息
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    
    # 针对双臂控制组进行控制，直接设置控制组的力矩命令
    hardware.setControlGroupFtCmd("left_manipulator",gravity_torque[group_joint_vel_order["left_manipulator"]])
    hardware.setControlGroupFtCmd("right_manipulator",gravity_torque[group_joint_vel_order["right_manipulator"]])
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)


In [ ]:
# 去使能机器人
hardware.disableRobot()